# Purpose

This is to connect with BQ and upload data to the datawarehouse

In [52]:
# install unique libraries to this project if needed
# pip install pandas-gbq
# pip install google-cloud-bigquery

# load in relevant libraries
import os
import pandas as pd
from google.cloud import bigquery
import pandas_gbq as pd_gbq

In [47]:
# store credentials and log into google cloud client
credentials_path = "C:/Users/Michael/OneDrive/Python/YouTube/youtube-video-data-328718-private.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials_path
gc_project = 'youtube-video-data-328718'
client = bigquery.Client(project=gc_project)

In [49]:
# create a new dataset (only needs to happen once)
dataset_id = '{}.youtubeVideoData'.format(client.project)
dataset = bigquery.Dataset(dataset_id)

# Send the dataset to the API for creation, with an explicit timeout.
# Raises google.api_core.exceptions.Conflict if the Dataset already
# exists within the project.
dataset = client.create_dataset(dataset, timeout=30)  # Make an API request.
print("Created dataset {}.{}".format(client.project, dataset.dataset_id))

Created dataset youtube-video-data-328718.youtubeVideoData


In [50]:
# create a new blank table (only needs to happen once)
# table_id = "your-project.your_dataset.your_table_name"
table_id = gc_project + '.' + 'youtubeVideoData.videoData'

# create table schema (Note the column names need to be exactly the same as the df columns)
schema = [
    bigquery.SchemaField("videoId", "STRING", mode = "NULLABLE"),
    bigquery.SchemaField("title", "STRING", mode = "NULLABLE"),
    bigquery.SchemaField("channelId", "STRING", mode = "NULLABLE"),
    bigquery.SchemaField("channelTitle", "STRING", mode = "NULLABLE"),
    bigquery.SchemaField("uploadTime", "TIMESTAMP", mode = "NULLABLE"),
]

# define table criteria and create
table = bigquery.Table(table_id, schema=schema)
table = client.create_table(table)  # Make an API request.
print(
    "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
)

Created table youtube-video-data-328718.youtubeVideoData.videoData


In [53]:
# Note: Could only get data to be uploaded via pandas_gbq and not google.cloud
# read data frame and change uploadTime to correct data format to allow upload

df = pd.read_csv('videoInfo.csv', encoding = 'utf-8-sig')
df.uploadTime = pd.to_datetime(df.uploadTime)

pd_gbq.to_gbq(df,
              'youtubeVideoData.videoData',
              project_id = gc_project,
              if_exists='replace') #options are 'fail', 'append', and 'replace'

1it [00:05,  5.90s/it]


In [57]:
# repeat creating a table for the parent comments data

# create a new blank table (only needs to happen once)
# table_id = "your-project.your_dataset.your_table_name"
table_id = gc_project + '.' + 'youtubeVideoData.parentComments'

# create table schema (Note the column names need to be exactly the same as the df columns)
schema = [
    bigquery.SchemaField("comments", "STRING", mode = "NULLABLE"),
    bigquery.SchemaField("commentsId", "STRING", mode = "NULLABLE"),
    bigquery.SchemaField("repliesCount", "INTEGER", mode = "NULLABLE"),
    bigquery.SchemaField("likesCount", "INTEGER", mode = "NULLABLE"),
    bigquery.SchemaField("videoId", "STRING", mode = "NULLABLE"),
]

# define table criteria and create
table = bigquery.Table(table_id, schema=schema)
table = client.create_table(table)  # Make an API request.
print(
    "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
)

Created table youtube-video-data-328718.youtubeVideoData.parentComments


In [59]:
# Note: Could only get data to be uploaded via pandas_gbq and not google.cloud
# read data frame and change uploadTime to correct data format to allow upload

df = pd.read_csv('parentComments.csv', encoding = 'utf-8-sig')

pd_gbq.to_gbq(df,
              'youtubeVideoData.parentComments',
              project_id = gc_project,
              if_exists='append') #options are 'fail', 'append', and 'replace'

1it [01:22, 82.43s/it]
